In [3]:
# Import necessary libraries
import os
import re
import nltk
import numpy as np
import pandas as pd
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
import time

# downloaded tokenizer 
nltk.download('punkt')

# Path to the main data folder
data_folder = 'Data'

# Initialize ISRIStemmer
stemmer = ISRIStemmer()

# Define the stopwords list 
arabic_stopwords = [
    'نيسان', 'هما', 'آه', 'بس', 'أخبر', 'لا', 'مه', 'كل', 'بَسْ', 'إنما', 'ترك', 'لعل', 'إلّا', 'قد', 'ألف', 
    'أنى', 'بك', 'وَيْ', 'لستما', 'خبَّر', 'ت', 'ذينك', 'أولالك', 'حيَّ', 'نا', 'هَجْ', 'ما', 'سرا', 'قلما', 
    'وما', 'اتخذ', 'عسى', 'يورو', 'عجبا', 'اللائي', 'إياها', 'جنيه', 'كلما', 'جيم', 'فاء', 'بهن', 'أولاء', 
    'سين', 'حادي', 'خمسين', 'ض', 'بماذا', 'حزيران', 'شباط', 'مع', 'غير', 'وا', 'كليكما', 'بات', 'ثلاثين', 
    'تِه', 'هَاتانِ', 'اثنان', 'ه', 'ثمة', 'ست', 'فو', 'لم', 'مائة', 'يفعلان', 'التي', 'زود', 'نعم', 'شمال', 
    'أسكن', 'ة', 'سحقا', 'ثان', 'ذين', 'تين', 'من', 'ا', 'رزق', 'أنتم', 'هؤلاء', 'تارة', 'عل', 'فمن', 'ثامن', 
    'كذلك', 'ثمانين', 'أوت', 'رُبَّ', 'إيه', 'لي', 'لبيك', 'ثاء', 'أمسى', 'زعم', 'دولار', 'ثلاثمائة', 'ذلك', 
    'هذان', 'ذات', 'بخ', 'لكنما', 'هنا', 'هناك', 'ح', 'لمّا', 'جميع', 'بعض', 'ستمائة', 'هَيْهات', 'لكي', 'خمسون', 
    'جويلية', 'لات', 'ريال', 'هذه', 'خ', 'لهما', 'عاد', 'لست', 'آمينَ', 'وإن', 'كأيّن', 'كلا', 'أقبل', 'إحدى', 
    'هل', 'اربعون', 'بئس', 'فيما', 'حجا', 'ثلاثون', 'كأنما', 'ثمانمئة', 'اثنا', 'الألى', 'أجمع', 'صبرا', 'كان', 
    'انقلب', 'مادام', 'لسن', 'تِي', 'عشر', 'نحو', 'فلان', 'بؤسا', 'ذهب', 'بطآن', 'أحد', 'عاشر', 'درى', 'الآن', 
    'لعمر', 'سوف', 'ّأيّان', 'ولكن', 'أول', 'أطعم', 'ألا', 'صاد', 'اللتيا', 'ء', 'مكانَك', 'كيت', 'صبر', 'ثلاثة', 
    'جوان', 'ياء', 'عن', 'عند', 'أينما', 'جانفي', 'لما', 'لئن', 'ذَيْنِ', 'فضلا', 'د', 'ءَ', 'كما', 'حيث', 
    'سادس', 'تانِ', 'سبعمئة', 'أصلا', 'ميم', 'مافتئ', 'أفريل', 'أوشك', 'أبدا', 'كيفما', 'إياه', 'إذن', 'ومن', 
    'أمام', 'واو', 'يوان', 'ع', 'تسعمائة', 'صهْ', 'مايو', 'لسنا', 'نوفمبر', 'ظاء', 'شَتَّانَ', 'مازال', 'خمسمئة', 
    'ين', 'إياكن', 'كى', 'أو', 'فإن', 'ن', 'ذانِ', 'ذِه', 'مكانكنّ', 'تلكم', 'أرى', 'ديسمبر', 'شبه', 'كثيرا', 
    'ثمنمئة', 'تعلَّم', 'غدا', 'غين', 'هَاتِه', 'يناير', 'كلاهما', 'نيف', 'جلل', 'ليسا', 'إياهن', 'اللتين', 
    'إليكم', 'دونك', 'كأنّ', 'عشرين', 'أيّ', 'ذلكم', 'أي', 'أربعاء', 'سابع', 'أل', 'إيهٍ', 'حتى', 'سبت', 
    'حبيب', 'خاء', 'هلّا', 'عامة', 'أيضا', 'كسا', 'أى', 'جمعة', 'هاتان', 'ب', 'لوما', 'اللتان', 'أغسطس', 
    'باء', 'إذما', 'وإذ', 'ص', 'عليه', 'تعسا', 'إمّا', 'ريث', 'قطّ', 'لو', 'أنت', 'ليست', 'ما برح', 'حين', 
    'ف', 'ضحوة', 'وراءَك', 'عما', 'كن', 'إلَيْكَ', 'لكنَّ', 'خلافا', 'عدا', 'لهن', 'بل', 'هيا', 'ارتدّ', 'أين', 
    'كرب', 'تسعة', 'نحن', 'تسعمئة', 'فيه', 'لن', 'أُفٍّ', 'إن', 'تفعلين', 'علق', 'هكذا', 'حدَث', 'هَذِه', 
    'هيت', 'كي', 'ك', 'صباح', 'وجد', 'حمٌ', 'كذا', 'أنتما', 'أنتِ', 'ستون', 'ستين', 'تلقاء', 'إياك', 'تموز', 
    'أهلا', 'حسب', 'إذ', 'عشرون', 'طَق', 'كانون', 'لكما', 'علم', 'اللذين', 'ثاني', 'ذواتا', 'أمد', 'رابع', 
    'س', 'لدن', 'شتان', 'عليك', 'كأين', 'أيلول', 'سبعمائة', 'فرادى', 'بغتة', 'قام', 'ؤ', 'أنًّ', 'بين', 'إنا', 
    'هاته', 'م', 'ضاد', 'تسعين', 'حاي', 'وهو', 'عَدَسْ', 'إليكنّ', 'طاق', 'مذ', 'بكم', 'همزة', 'ثم', 'بعدا', 
    'إنه', 'والذين', 'فبراير', 'سبعون', 'أيار', 'هنالك', 'آهٍ', 'منذ', 'آها', 'أبٌ', 'راح', 'أولئك', 'بلى', 
    'تبدّل', 'تسع', 'سبتمبر', 'لا سيما', 'ليرة', 'كلَّا', 'سبعة', 'ذيت', 'حرى', 'له', 'ثمانية', 'سبحان', 'مئة', 
    'اثني', 'هاكَ', 'كاد', 'أمامك', 'استحال', 'أعطى', 'هاء', 'خال', 'جير', 'أبريل', 'ذا', 'شيكل', 'قبل', 
    'كِخ', 'الذين', 'بمن', 'غ', 'تفعلون', 'ثالث', 'كم', 'مما', 'أربعمائة', 'ئ', 'تانِك', 'وإذا', 'ش', 'تلكما', 
    'آذار', 'لكيلا', 'هيّا', 'كيف', 'غالبا', 'لكم', 'إلى', 'خميس', 'هَذِي', 'ته', 'أما', 'في', 'كأي', 'إليكَ', 
    'هللة', 'خاصة', 'أخذ', 'ثلاثمئة', 'ذِي', 'خلا', 'إذا', 'خلف', 'صار', 'ما أفعله', 'يونيو', 'ولو', 'شين', 
    'ذي', 'آنفا', 'بنا', 'ثماني', 'لستم', 'تاء', 'بيد', 'إليك', 'ذلكما', 'كلتا', 'هاك', 'آ', 'مكانكما', 
    'آناء', 'أوّهْ', 'ظ', 'ماي', 'أنشأ', 'سمعا', 'اللاتي', 'نبَّا', 'لستن', 'أكثر', 'أن', 'بهما', 'أفٍّ', 
    'تجاه', 'اللذان', 'كاف', 'هَذَيْنِ', 'سنتيم', 'بما', 'ط', 'هبّ', 'آض', 'لها', 'أقل', 'ولا', 'لاسيما', 
    'لعلَّ', 'حمدا', 'عيانا', 'صهٍ', 'مارس', 'نون', 'قاف', 'مئتان', 'خمس', 'أخٌ', 'هَذانِ', 'فلا', 'وهب', 
    'مرّة', 'ى', 'فيم', 'ليت', 'خمسة', 'نَخْ', 'خامس', 'ستة', 'ذواتي', 'ثمَّ', 'أصبح', 'منه', 'الذي', 'إنَّ', 
    'ذانك', 'حَذارِ', 'أ', 'سبع', 'هَاتِي', 'هو', 'لولا', 'الألاء', 'ليستا', 'أربع', 'لنا', 'هذي', 'رجع', 
    'درهم', 'على', 'إما', 'شتانَ', 'تحوّل', 'حاء', 'أجل', 'آهاً', 'ج', 'كلّما', 'ممن', 'اربعين', 'تينك', 
    'إليكما', 'م', 'إذاً',"اذا", 'سرعان', 'سقى', 'تخذ', 'أبو', 'أمامكَ', 'هي', 'إيانا', 'هَؤلاء', 'بسّ', 'ذال', 
    'يفعلون', 'عدَّ', 'آهِ', 'ما انفك', 'عين', 'و', 'قاطبة', 'أنّى', 'أربعة', 'راء', 'دون', 'هاتي', 'ها', 
    'منها', 'ثمّ', 'أنتن', 'واهاً', 'بها', 'سوى', 'ر', 'ثلاثاء', 'طالما', 'ابتدأ', 'يوليو', 'مليم', 'رويدك', 
    'أيها', 'هلم', 'إياهم', 'أمّا', 'هاهنا', 'ذ', 'هيهات', 'هَاتَيْنِ', 'غداة', 'اللواتي', 'لدى', 'ق', 
    'ساء', 'ثمانون', 'ألفى', 'دينار', 'بكن', 'بَلْهَ', 'أعلم', 'تفعلان', 'أخو', 'صراحة', 'بكما', 'أنا', 
    'إياكما', 'تَيْنِ', 'هلا', 'أنبأ', 'واحد', 'دال', 'كأن', 'هاتين', 'تسعون', 'مساء', 'مهما', 'زاي', 'ليسوا', 
    'إياهما', 'يمين', 'اثنين', 'عوض', 'ظنَّ', 'حيثما', 'ذاك', 'أيا', 'علًّ', 'رأى', 'لام', 'طفق', 'بهم', 
    'ليس', 'كليهما', 'ستمئة', 'أمس', 'ظلّ', 'كأيّ', 'حمو', 'آي', 'أم', 'تاسع', 'صدقا', 'آب', 'انبرى', 
    'هذين', 'فيها', 'أيّان', 'ذه', 'متى', 'والذي', 'تي', 'هن', 'عشرة', 'طرا', 'حاشا', 'إياي', 'فلس', 'ورد', 
    'فيفري', 'أكتوبر', 'حار', 'أربعمئة', 'سبعين', 'مكانكم', 'مثل', 'قرش', 'تحت', 'به', 'لكن', 'غادر', 'ي', 
    'بعد', 'لهم', 'إياكم', 'إليكن', 'تلك', 'ز', 'ل', 'إى', 'نَّ', 'أف', 'طاء', 'هم', 'هَذا', 'ثلاث', 'ذلكن', 
    'إزاء', 'ذو', 'حبذا', 'ثمان', 'نفس', 'ثمّة', 'معاذ', 'حقا', 'لك', 'تشرين', 'دواليك', 'اخلولق', 'ذوا', 
    'بضع', 'فوق', 'فإذا', 'شرع', 'ث', 'إي', 'ذان', 'أوه', 'إلا', 'بي', 'أفعل به', 'يا', 'خمسمائة', 'وُشْكَانَ', 
    'جعل', 'بخٍ', 'أضحى', 'هذا'
]
# Function to load texts from folders
def load_texts_from_folders(data_folder):
    texts = []
    labels = []
    
    # List of categories folders
    categories = ['articlesEconomy', 'articlesLocal', 'articlesInternational', 'articlesSports', 'articlesReligion', 'articlesCulture']
    
    # Loop through the folders for each category
    for category in categories:
        folder_path = os.path.join(data_folder, category)
        
        # Load text files from each category
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            
            # Read the contents of each file
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                texts.append(text)
                labels.append(category)
    
    return texts, labels

# Function to remove stopwords using the predefined list
def remove_stopwords_arabic(text):
    words = text.split()
    # Remove stopwords using the stopwords list
    filtered_words = [word for word in words if word not in arabic_stopwords]
    return ' '.join(filtered_words)

# Function to normalize Arabic texts
def normalize_arabic_text(text):
    # Replace "صلى الله عليه وسلم" with a single token
    text = text.replace("صلى الله عليه وسلم", "صلى_الله_عليه_وسلم")
    
    # Remove diacritics 
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    
    # Remove both Arabic and English numbers
    text = re.sub(r'[0-9٠-٩]+', '', text)
    
    # Remove all types of Hamza
    text = re.sub(r'[إأآءؤئ]', 'ا', text)
    
    # Remove special symbols and punctuation marks
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Function to apply stemming using ISRIStemmer
def apply_stemming(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Function to tokenize the text
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

# Hand-crafted features - only two features
def extract_handcrafted_features(text):
    tokens = word_tokenize(text)
    
    # Feature 1: Number of occurrences of economy related words
    economy_words = ["مال", "اقتصاد", "سوق", "تجارة"]
    num_economy = sum(1 for word in tokens if word in economy_words)
    
    # Feature 2: Number of unique characters
    num_unique_chars = len(set(text))
    
    return [num_economy, num_unique_chars]

# Function to select the top 1000 features
def select_top_1000_features(X, y, num_features=1000):
    # Ensure that the number of selected features does not exceed 1000
    k = min(num_features, X.shape[1])
    
    # Apply feature selection using Chi-Squared
    selector = SelectKBest(score_func=chi2, k=k)
    X_new = selector.fit_transform(X, y)
    
    return X_new

# Add a time counter
start_time = time.time()

# Load the texts
print("Loading texts...")
texts, labels = load_texts_from_folders(data_folder)
print(f"Loaded in {time.time() - start_time:.2f} seconds")

# Clean the texts and remove stopwords
start_time = time.time()
print("Cleaning texts...")
cleaned_texts = [remove_stopwords_arabic(text) for text in texts]
print(f"Cleaned in {time.time() - start_time:.2f} seconds")

# Apply normalization to the cleaned texts
start_time = time.time()
print("Applying normalization...")
normalized_texts = [normalize_arabic_text(text) for text in cleaned_texts]
print(f"Normalized in {time.time() - start_time:.2f} seconds")

# Apply stemming to the normalized texts
start_time = time.time()
print("Applying stemming...")
stemmed_texts = [apply_stemming(text) for text in normalized_texts]
print(f"Stemmed in {time.time() - start_time:.2f} seconds")

# Convert texts into tokens
start_time = time.time()
print("Tokenizing texts...")
tokenized_texts = [tokenize_text(text) for text in stemmed_texts]
print(f"Tokenized in {time.time() - start_time:.2f} seconds")

# Convert the texts to numerical representation using TF-IDF
start_time = time.time()
print("Converting texts to TF-IDF...")
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform([' '.join(tokens) for tokens in tokenized_texts])
print(f"Converted to TF-IDF in {time.time() - start_time:.2f} seconds")

# Extract handcrafted features (only two features)
start_time = time.time()
print("Extracting handcrafted features...")
handcrafted_features = np.array([extract_handcrafted_features(text) for text in cleaned_texts])
print(f"Handcrafted features extracted in {time.time() - start_time:.2f} seconds")

# Combine TF-IDF features with handcrafted features
start_time = time.time()
print("Combining features...")
X_combined = np.hstack((X_tfidf.toarray(), handcrafted_features))
print(f"Combined in {time.time() - start_time:.2f} seconds")

# Select the top 1000 features
start_time = time.time()
print("Selecting top 1000 features...")
X_new = select_top_1000_features(X_combined, labels, num_features=1000)
print(f"Selected top features in {time.time() - start_time:.2f} seconds")

# Save the processed data to a CSV file
start_time = time.time()
print("Saving data to CSV...")
df = pd.DataFrame(X_new)
df['class'] = labels
df.to_csv('processed_data.csv', index=False)
print(f"Data saved to CSV in {time.time() - start_time:.2f} seconds")

print("Process completed successfully!")
                              





[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading texts...
Loaded in 9.35 seconds
Cleaning texts...
Cleaned in 48.37 seconds
Applying normalization...
Normalized in 5.80 seconds
Applying stemming...
Stemmed in 36.07 seconds
Tokenizing texts...
Tokenized in 13.75 seconds
Converting texts to TF-IDF...
Converted to TF-IDF in 4.72 seconds
Extracting handcrafted features...
Handcrafted features extracted in 27.86 seconds
Combining features...
Combined in 17.72 seconds
Selecting top 1000 features...
Selected top features in 5.43 seconds
Saving data to CSV...
Data saved to CSV in 12.13 seconds
Process completed successfully!


In [4]:
# Import necessary libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X_combined, labels, test_size=0.3, random_state=1)

# Apply SelectKBest to select the top 1000 features using Chi-Squared 
selector = SelectKBest(score_func=chi2, k=1000)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Define the scaler and scale the selected training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Define and train the SVM model with polynomial kernel
svm_model = SVC(C=1.0, kernel='poly', degree=1, tol=1e-12, random_state=1)
svm_model.fit(X_train_scaled, y_train)

# Evaluate the SVM model on the test set
y_pred = svm_model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))




Accuracy: 0.9275150629906883
Classification Report:
                        precision    recall  f1-score   support

      articlesCulture       0.91      0.91      0.91       735
      articlesEconomy       0.87      0.90      0.88       964
articlesInternational       0.95      0.93      0.94       481
        articlesLocal       0.85      0.85      0.85      1015
     articlesReligion       0.98      0.99      0.99      1051
       articlesSports       0.99      0.98      0.98      1231

             accuracy                           0.93      5477
            macro avg       0.93      0.92      0.93      5477
         weighted avg       0.93      0.93      0.93      5477



In [8]:
# Function to predict the class of a new text file
def predict_text_class(file_path, svm_model, vectorizer, scaler, selector):
    # Load the new text file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Preprocess the text (same steps as used in training)
    cleaned_text = remove_stopwords_arabic(text)
    normalized_text = normalize_arabic_text(cleaned_text)
    stemmed_text = apply_stemming(normalized_text)
    tokenized_text = ' '.join(tokenize_text(stemmed_text))
    
    # Convert the processed text to numerical representation using TF-IDF
    text_tfidf = vectorizer.transform([tokenized_text]).toarray()
    
    # Extract handcrafted features for the new text
    handcrafted_features = np.array(extract_handcrafted_features(cleaned_text)).reshape(1, -1)
    
    # Combine TF-IDF features with handcrafted features
    combined_features = np.hstack((text_tfidf, handcrafted_features))
    
    # Apply the same feature selection that was used during training
    combined_features_selected = selector.transform(combined_features)
    
    # Normalize the features 
    combined_features_scaled = scaler.transform(combined_features_selected)
    
    # Predict the class using the trained SVM model
    predicted_class = svm_model.predict(combined_features_scaled)
    
    return predicted_class[0]

# Example usage
file_path = r'C:\Users\DELL\Desktop\NLP-assignment\sp.txt'  # Provide the path to the text file
predicted_class = predict_text_class(file_path, svm_model, tfidf_vectorizer, scaler, selector)

print(f"The predicted class for the given text is: {predicted_class}")





The predicted class for the given text is: articlesSports


In [6]:
import pickle

# Save the SVM model, vectorizer, scaler, and selector to a file
with open('svm_model.pkl', 'wb') as file:
    pickle.dump((svm_model, tfidf_vectorizer, scaler, selector), file)

print("Model saved successfully!")


Model saved successfully!
